# Инструкция по разработке чат-бота Вконтакте
#### Цель 
Разработать простого чат-бота для сообщества социальной сети Вконтакте средствами языка программирования Python и VK API, изучить документацию API ВКонтакте.

Документация по API Вконтакте - https://vk.com/dev/first_guide

#### Задачи
1.	Настроить IDE PyCharm.
2.	Настроить сообщество и получить API-ключ (токен).
3.	Настроить Long Poll соединение.
4.	Определить команды, которые будет обрабатывать бот.
5.	Добавить клавиатуру для общения с ботом.

#### Software stack
* *IDE PyCharm*
* *Python 3.6*
* *vk-api*
* *Long Poll API*


Итоговый проект доступен в репозитории Github по ссылке https://github.com/dariabokareva/python-vk-chatbot. 


### 1.	Настройка IDE PyCharm
Для разработки чат-бота используется язык программирования **_Python 3.6_** и среда **_JetBrains PyCharm Community Edition 2019.1.3._** Проект поддерживается также версией Python 3.7.

Скачать и установить Python необходимой версии можно по ссылке https://www.python.org/downloads/. 

IDE PyCharm Community Edition расположена по ссылке https://www.jetbrains.com/ru-ru/pycharm/download/.

После установки Python и PyCharm необходимо запустить среду разработки и создать новый проект (File > Create Project).

Для разработки бота потребуется установить дополнительные модули. Это можно сделать в настройках проекта (File > Settings > Project > Project Inetpreter > +). Необходимо найти модуль **_vk-api_** в списке пакетов и установить (Install Package).

### 2.	Настройка сообщества и получение API-ключа (токена)
Для подключения бота к сообществу Вконтакте необходимо получить ключ доступа. Для этого необходимо перейти в свою группу Вконтакте или создать её. Затем в правом меню: Управление > Работа с API > Ключи доступа > Создать ключ. Боту необходимо дать разрешение на доступ к сообщениям.


Далее необходимо перейти на вкладку Long Poll API и включить его (версия по умолчанию 5.50).
Также на вкладке Управление > Настройки для бота, необходимо включить «Возможности ботов» и поставить галочку на пункте «Добавить кнопку “Начать”».
Сообщество готово к подключению бота.


### 3.	Настройка Long Poll соединения
Для написания кода необходимо создать в проекте Python File (файл с разрешением «.py»).

Необходимо создать 3 файла: **config.py**, **server.py** и **server_manager.py**. Далее открываем файл **config.py**. В нём будет хранится токен. Создаём переменную *vk_api_token* и копируем в неё ключ (строку).


In [1]:
vk_api_token = "your token here"

Открываем файл **server.py**. В этом файле будет код подключения и работы с Long poll API. Первым делом необходимо импортировать библиотеки, которые понадобятся для работы.

In [7]:
import vk_api
import vk_api.vk_api
import random
from vk_api.longpoll import VkLongPoll
from vk_api.longpoll import VkEventType

Создаём класс *Server* c функцией __ init __().

In [5]:
class Server:

    def __init__(self, api_token, server_name: str = "Empty"):
        self.server_name = server_name
        # Для Long Poll
        self.vk = vk_api.VkApi(token=api_token)

        # Для использования Long Poll API
        self.long_poll = VkLongPoll(self.vk)

        # Для вызова методов vk_api
        self.vk_api = self.vk.get_api()

        self.users = {}

Функции и методы в ЯП Python создаются с помощью ключевого слова «def».

Функция __ init __ содержит следующие аргументы:

* *self* – аналог «this» в других языках программирования. Self ссылается на экземпляр класса для которого вызывается метод. В методе __ init __ self ссылается на только что созданный объект. При вызове метода self не указывается, Python добавит его автоматически;
* *api_token* – аргумент, в который передаётся ключ доступа.
* *server_name* – аргумент, принимающий имя сервера, по умолчанию “Empty”

В классе *Server* создадим также функцию send_message().


In [8]:
def send_message(self, send_id, message, keyboard):
    """
    Отправка сообщения через метод messages.send
    :param send_id: vk id пользователя, который получит сообщение
    :param message: содержимое отправляемого письма
    :param keyboard: клавиатура для пользователя
    :return: None
    """
    return self.vk_api.messages.send(user_id=send_id,
                                     message=message,
                                     random_id=random.randint(0, 2048),
                                     keyboard=keyboard)


Данная функция возвращает сообщение, которое будет отправлено пользователю, при помощи метода vk_api.messages.send().
Обязательные именованные аргументы метода messages.send():
* *user_id* – принимает id пользователя, которому необходимо отправить сообщение;
* *message* – содержимое сообщения;
* *random_id* – идентификатор, позволяющий ограничивать повторную отправку сообщения пользователю. Необходимо генерировать рандомное число с помощью библиотеки random.

Аргумент *keyboard* является необязательным, но будет необходим для отправки пользователю клавиатуры с командами.

Создаём функцию start(), при вызове которой бот начнёт отслеживать сообщения через Long Poll и вызывать метод send_message() для отправки ответа пользователю.


In [11]:
def start(self):
    for event in self.long_poll.listen():  # Слушаем сервер
        # Пришло новое сообщение
        if event.type == VkEventType.MESSAGE_NEW and event.text:
            user_id = event.user_id
            kbd = open("keyboard.json", "r", encoding="UTF-8").read()
            self.send_message(user_id, message_out, kbd)

Во избежание ошибок при запуске промежуточной версии необходимо создать в папке проекта файл keyboard.json (это можно сделать по аналогии с созданием Python File или через проводник системы). Нужно прописать в нём следующий код:

{"buttons":[],"one_time":true}

Далее необходимо открыть файл **server_manager.py**, и прописать управление сервером. Для этого необходимо импортировать в него класс *Server* из файла **server.py** и переменную *vk_api_token* из **config.py**.
Создаём переменную *server1* как объект класса Server и передаём необходимые параметры.


При запуске файла **server_manager.py** (Главное меню > Run > Run…) бот начинает работать. 

### 4.	Определение команд, которые будет обрабатывать чат-бот
Логика бота:

«Слушаем» сервер Long Poll > получаем сообщение пользователя > Передаем сообщение в Commander.input() > Определяем команду > Возвращаем ответ > Передаем пользователю. Переходим к реализации.

Для создания команд, которые сможет прочитать бот, необходимо создать файл **command_enum.py** и импортировать в него модуль **_enum_**, реализующий перечисления. Далее нужно создать класс *Command(Enum)*.

In [12]:
from enum import Enum


class Command(Enum):
    """Класс, содержащий перечисление команд"""

    """Начать"""
    start = ["Начать", "начать", "start", "Start"]

По умолчанию у пользователя при первом сообщении на клавиатуре будет доступна кнопка «Начать» (это функционал Вконтакте), при её нажатии боту отправляется одноимённое сообщение. Поэтому добавим в класс комманду start, которая будет определяться при вводе пользователем любого сообщения, перечисленного в массиве.

Здесь же создадим класс *Message(Enum)*, в котором будет содержаться перечисление сообщений, которые бот сможет отправлять в ответ.

In [13]:
class Message(Enum):
    """Класс, содержащий перечисление сообщений для каждой команды"""

    start_msg = 'вот, что я могу подсказать:\n\n1&#8419; Информацию о мероприятии\n2&#8419; Возраст участников\n' \
                '3&#8419; Стоимость участия\n4&#8419; Расписание\n\nВыбери интересующую тебя команду!'

Наборы символов «1&#8419;» в строке – это коды смайлов Вконтакте, именно так бот сможет отправлять в ответ сообщения со смайлами (например, «&#128522;	» - это смайлик «улыбка»).

Теперь необходимо создать файл commander.py, который будет обрабатывать команды. Импортируем в него классы из command_enum. А также модуль re, который пригодится для более точной обработки текста.

Создадим также класс *Commander* с функциями __ init __ (self) и input(self, msg).


In [ ]:
from command_enum import Command, Message
import re


class Commander:

    def __init__(self):

        # Для запоминания последней команды
        self.last_command = None

        # Для запомминания ответов пользователя
        self.last_ans = None

    def input(self, msg):
        """
        Функция принимающая сообщения пользователя
        :param msg: Сообщение
        :return: Ответ пользователю, отправившему сообщение
        """
        # if msg in Command.start.value:
        if re.search('начать', msg):
            self.last_command = Command.start
            return Message.start_msg.value
        
        return 'Я не знаю такой команды &#128532;'

Функция input содержит аргумент msg, который принимает сообщение пользователя. 

Далее осуществляется проверка на соответствие сообщения команде Command.start.value.

Её можно реализовать двумя способами (один из них закомментирован). Оптимально, в данном случае, использовать функцию re.search(). Она осуществляет поиск по заданному слову (“начать”, в сообщении пользователя msg). Если слово содержится в сообщении в любом виде, то команда определяется как start.

Если использовать другой способ, необходимо прописывать все возможные варианты сообщения в команде start. Например, пользователь может написать «Начать.» или «начать:)», и, если их не будет в перечислении – бот не поймёт сообщение.
И так, если мы определили команду, как start, присваиваем переменной self.last_command значение Command.start.value и возвращаем ответ пользователю (Message.start_msg.value).


Если пользователь отправил нам иное сообщение, возвращаем сообщение о том, что бот незнаком с такой командой.
Обработка команды готова, теперь необходимо отредактировать файл server.py.

Перепишем функцию start() в соответствие с указанной раннее логикой бота.

In [ ]:
def start(self):
    for event in self.long_poll.listen():  # Слушаем сервер
        # Пришло новое сообщение
        if event.type == VkEventType.MESSAGE_NEW and event.to_me and event.text:
            if event.user_id not in self.users:
                self.users[event.user_id] = Commander()

            user_id = event.user_id
            username = self.get_user_name(user_id)
            message_out = self.users[event.user_id].input(event.text.lower())
            kbd = open("keyboard.json", "r", encoding="UTF-8").read()

            if message_out == Message.start_msg.value:
                message_out = username + ", " + Message.start_msg.value
            self.send_message(user_id, message_out, kbd)

В класс *Server* добавим функцию get_user_name(user_id), чтобы получить имя пользователя по id и обратиться к нему при отправке сообщения.

In [ ]:
def get_user_name(self, user_id):
    """ Функция, возвращающая имя пользователя"""
    user_name = self.vk_api.users.get(user_id=user_id)[0]['first_name']

    return user_name

### 5.	Добавление клавиатуры пользователя
Последний шаг – добавление клавиатуры пользователя. В коде уже прописана её реализация, осталось только отредактировать файл keyboard.json.

Пример реализации кнопок на клавиатуре (код в keyboard.json):


{
  "one_time": false,
  "buttons": [
    [{
      "action": {
        "type": "text",
        "label": "&#128293; О мероприятии",
        "payload": "{\"button\": \"1\"}"
      },
      "color": "positive"
    },
      {
        "action": {
          "type": "text",
          "label": "&#128118; Возраст",
          "payload": "{\"button\": \"1\"}"
        },
        "color": "positive"
      }],
    [{
        "action": {
          "type": "text",
          "label": "&#128176; Цена",
          "payload": "{\"button\": \"2\"}"
        },
        "color": "positive"
      },
      {
        "action": {
          "type": "text",
          "label": "&#128197; Расписание",
          "payload": "{\"button\": \"2\"}"
        },
        "color": "positive"
      }],
    [{
       "action": {
          "type": "text",
          "label": "Хочу записаться&#128165;\t",
          "payload": "{\"button\": \"2\"}"
        },
        "color": "positive"
    }]
  ]
}



Каждая команда, добавленная на клавиатуру, должна быть обработана в файлах **command_enum.py**, **commander.py**, **server.py** по аналогии с командой начать.

Теперь запустим файл **server_manager.py** и напишем в наше сообщество сообщение «Начать» (или нажнём на кнопку «Начать», которая была включена раннее).

Чтобы чат-бот был всегда онлайн, его следует разместить на хостинге (например, https://www.pythonanywhere.com/)

Исходный код проекта доступен в Github-репозитории https://github.com/dariabokareva/python-vk-chatbot.  
